# Sample codes

Install packages

In [ ]:
#!pip install dowhy

In [1]:
import pdb
import numpy as np
import pandas as pd

import pickle
from matplotlib import pyplot as plt

import dowhy
import dowhy.datasets
from dowhy import CausalModel

import networkx as nx

## Load data file

Data file (Tabular format)

In [ ]:
data_path = 'data-dir/' #directory where causal structure csv and data file are saved

In [ ]:
data = pd.read_csv(data_path+'occurrence_sample.txt')

## Load causal structure

Load causal structure file. The file format is csv with headers=(Cause, Effect, #Appearance)

In [4]:
causal_graph='bayesian-network-sample'
df = pd.read_csv(data_path+causal_graph+'.csv')

For robustness of Bayesian network, we repeated the causal structure learning ten times. We will focus on directed edges that appears more than `threshold` times in ten repetitions

In [5]:
threshold=5

Create `networkx` object

In [6]:
G=nx.from_pandas_edgelist(df[df['# Apperance']>=threshold], source='Cause',target='Effect')
nx.write_gml(G,data_path+causal_graph+'.gml')

## Define potential outcome model

In [8]:
treatment = 'v_296.2' #depression
outcome='v_alzheimer' #ADRD

data[treatment]=data[treatment].astype(bool)
data[outcome]=data[outcome].astype(bool)

In [9]:
model=CausalModel(data=data,
                 treatment=treatment,
                 outcome=outcome,
                 graph=data_path+causal_graph+'.gml')

INFO:dowhy.causal_graph:If this is observed data (not from a randomized experiment), there might always be missing confounders. Adding a node named "Unobserved Confounders" to reflect this.
INFO:dowhy.causal_model:Model to find the causal effect of treatment ['v_296.2'] on outcome ['v_alzheimer']


Visualize the causal structure

In [ ]:
#model.view_model()

In [10]:
identified_estimand=model.identify_effect(proceed_when_unidentifiable=True)

INFO:dowhy.causal_identifier:Common causes of treatment and outcome:['v_280.1', 'v_722.6', 'v_292.3', 'v_401.22', 'v_Female', 'v_vascular_disease', 'v_443.9', 'v_386.9', 'v_789', 'v_495', 'v_411.2', 'v_366', 'v_433.21', 'v_782.3', 'v_heart_disease', 'v_300.1', 'v_433.8', 'v_350.2', 'v_916', 'v_530.11', 'U', 'v_585.31', 'v_425.1', 'v_585.1', 'v_339', 'v_332', 'v_452.2', 'v_obesity', 'v_261.4', 'v_290.3', 'v_1009', 'v_707.2', 'v_286.2', 'v_788', 'v_532', 'v_585.32', 'v_743.11', 'v_365', 'v_496.21', 'v_250.3', 'v_418', 'v_600', 'v_773', 'v_465', 'v_562.1', 'v_563', 'v_285', 'v_272.13', 'v_783', 'v_292.4', 'v_745', 'v_276.12', 'v_496', 'v_480', 'v_457.3', 'v_433.1', 'v_427.3', 'v_smoke', 'v_561.1', 'v_hypertension', 'v_760', 'v_721.1', 'v_586', 'v_head_injury', 'v_272.1', 'v_427.21', 'v_290.1', 'v_244.4', 'v_276.5', 'v_426.91', 'v_714.1', 'v_743.9', 'v_512.9', 'v_740.1', 'v_585.33', 'v_185', 'v_274.1', 'v_1002', 'v_585.3', 'v_512.8', 'v_707.1', 'v_591', 'v_272.11', 'v_395.1', 'v_174.11', '

In [11]:
print(identified_estimand)

Estimand type: nonparametric-ate
### Estimand : 1
Estimand name: backdoor
Estimand expression:
    d                                                                         
──────────(Expectation(v_alzheimer|v_280.1,v_722.6,v_292.3,v_401.22,v_Female,v
d[v_296.2]                                                                    

                                                                              
_vascular_disease,v_443.9,v_386.9,v_789,v_495,v_411.2,v_366,v_433.21,v_782.3,v
                                                                              

                                                                              
_heart_disease,v_300.1,v_433.8,v_350.2,v_916,v_530.11,v_585.31,v_425.1,v_585.1
                                                                              

                                                                              
,v_339,v_332,v_452.2,v_obesity,v_261.4,v_290.3,v_1009,v_707.2,v_286.2,v_788,v_
                                 

## Estimate causal effect

In [36]:
estimate=model.estimate_effect(identified_estimand,
                              method_name='backdoor.propensity_score_matching', target_units='atc') #target_units='ate' for ATE, 'att' for ATT

INFO:dowhy.causal_estimator:INFO: Using Propensity Score Matching Estimator
INFO:dowhy.causal_estimator:b: v_alzheimer~v_296.2+v_280.1+v_722.6+v_292.3+v_401.22+v_Female+v_vascular_disease+v_443.9+v_386.9+v_789+v_495+v_411.2+v_366+v_433.21+v_782.3+v_heart_disease+v_300.1+v_433.8+v_350.2+v_916+v_530.11+v_585.31+v_425.1+v_585.1+v_339+v_332+v_452.2+v_obesity+v_261.4+v_290.3+v_1009+v_707.2+v_286.2+v_788+v_532+v_585.32+v_743.11+v_365+v_496.21+v_250.3+v_418+v_600+v_773+v_465+v_562.1+v_563+v_285+v_272.13+v_783+v_292.4+v_745+v_276.12+v_496+v_480+v_457.3+v_433.1+v_427.3+v_smoke+v_561.1+v_hypertension+v_760+v_721.1+v_586+v_head_injury+v_272.1+v_427.21+v_290.1+v_244.4+v_276.5+v_426.91+v_714.1+v_743.9+v_512.9+v_740.1+v_585.33+v_185+v_274.1+v_1002+v_585.3+v_512.8+v_707.1+v_591+v_272.11+v_395.1+v_174.11+v_395.2+v_285.21+v_401.1+v_250.2+v_428.1+v_798+v_785+v_716.9+v_426.7+v_740.9+v_diabetes+v_318+v_452+v_1010+v_512.7+v_433.31+v_276.14+v_208+v_411.4+v_153.2
/home/anaconda3/lib/python3.7/site-packages/s

In [37]:
print(estimate)

*** Causal Estimate ***

## Identified estimand
Estimand type: nonparametric-ate
### Estimand : 1
Estimand name: backdoor
Estimand expression:
    d                                                                         
──────────(Expectation(v_alzheimer|v_280.1,v_722.6,v_292.3,v_401.22,v_Female,v
d[v_296.2]                                                                    

                                                                              
_vascular_disease,v_443.9,v_386.9,v_789,v_495,v_411.2,v_366,v_433.21,v_782.3,v
                                                                              

                                                                              
_heart_disease,v_300.1,v_433.8,v_350.2,v_916,v_530.11,v_585.31,v_425.1,v_585.1
                                                                              

                                                                              
,v_339,v_332,v_452.2,v_obesity,v_261.4,v_290.3,v_1009,v_707.2,v_

In [38]:
estimate.get_confidence_intervals()

INFO:dowhy.causal_estimator:INFO: The sample size: 13460
INFO:dowhy.causal_estimator:INFO: The number of simulations: 100
INFO:dowhy.causal_estimator:INFO: Using Propensity Score Matching Estimator
INFO:dowhy.causal_estimator:b: v_alzheimer~v_296.2+v_280.1+v_722.6+v_292.3+v_401.22+v_Female+v_vascular_disease+v_443.9+v_386.9+v_789+v_495+v_411.2+v_366+v_433.21+v_782.3+v_heart_disease+v_300.1+v_433.8+v_350.2+v_916+v_530.11+v_585.31+v_425.1+v_585.1+v_339+v_332+v_452.2+v_obesity+v_261.4+v_290.3+v_1009+v_707.2+v_286.2+v_788+v_532+v_585.32+v_743.11+v_365+v_496.21+v_250.3+v_418+v_600+v_773+v_465+v_562.1+v_563+v_285+v_272.13+v_783+v_292.4+v_745+v_276.12+v_496+v_480+v_457.3+v_433.1+v_427.3+v_smoke+v_561.1+v_hypertension+v_760+v_721.1+v_586+v_head_injury+v_272.1+v_427.21+v_290.1+v_244.4+v_276.5+v_426.91+v_714.1+v_743.9+v_512.9+v_740.1+v_585.33+v_185+v_274.1+v_1002+v_585.3+v_512.8+v_707.1+v_591+v_272.11+v_395.1+v_174.11+v_395.2+v_285.21+v_401.1+v_250.2+v_428.1+v_798+v_785+v_716.9+v_426.7+v_740.9+v

/home/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
INFO:dowhy.causal_estimator:INFO: Using Propensity Score Matching Estimator
INFO:dowhy.causal_estimator:b: v_alzheimer~v_296.2+v_280.1+v_722.6+v_292.3+v_401.22+v_Female+v_vascular_disease+v_443.9+v_386.9+v_789+v_495+v_411.2+v_366+v_433.21+v_782.3+v_heart_disease+v_300.1+v_433.8+v_350.2+v_916+v_530.11+v_585.31+v_425.1+v_585.1+v_339+v_332+v_452.2+v_obesity+v_261.4+v_290.3+v_1009+v_707.2+v_286.2+v_788+v_532+v_585.32+v_743.11+v_365+v_496.21+v_250.3+v_418+v_600+v_773+v_465+v_562.1+v_563+v_285+v_272.13+v_783+v_292.4+v_745+v_276.12+v_496+v_480+v_457.3+v_433.1+v_427.3+v_smoke+v_561.1+v_hypertension+v_760+v_721.1+v_586+v_head_injury+v_272.1+v_427.21+v_290.1+v_244.4+v_276.5+v_426.91+v_714.1+v_743.9+v_512.9+v_740.1+v_585.33+v_185+v_274.1+v_1

/home/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
INFO:dowhy.causal_estimator:INFO: Using Propensity Score Matching Estimator
INFO:dowhy.causal_estimator:b: v_alzheimer~v_296.2+v_280.1+v_722.6+v_292.3+v_401.22+v_Female+v_vascular_disease+v_443.9+v_386.9+v_789+v_495+v_411.2+v_366+v_433.21+v_782.3+v_heart_disease+v_300.1+v_433.8+v_350.2+v_916+v_530.11+v_585.31+v_425.1+v_585.1+v_339+v_332+v_452.2+v_obesity+v_261.4+v_290.3+v_1009+v_707.2+v_286.2+v_788+v_532+v_585.32+v_743.11+v_365+v_496.21+v_250.3+v_418+v_600+v_773+v_465+v_562.1+v_563+v_285+v_272.13+v_783+v_292.4+v_745+v_276.12+v_496+v_480+v_457.3+v_433.1+v_427.3+v_smoke+v_561.1+v_hypertension+v_760+v_721.1+v_586+v_head_injury+v_272.1+v_427.21+v_290.1+v_244.4+v_276.5+v_426.91+v_714.1+v_743.9+v_512.9+v_740.1+v_585.33+v_185+v_274.1+v_1

/home/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
INFO:dowhy.causal_estimator:INFO: Using Propensity Score Matching Estimator
INFO:dowhy.causal_estimator:b: v_alzheimer~v_296.2+v_280.1+v_722.6+v_292.3+v_401.22+v_Female+v_vascular_disease+v_443.9+v_386.9+v_789+v_495+v_411.2+v_366+v_433.21+v_782.3+v_heart_disease+v_300.1+v_433.8+v_350.2+v_916+v_530.11+v_585.31+v_425.1+v_585.1+v_339+v_332+v_452.2+v_obesity+v_261.4+v_290.3+v_1009+v_707.2+v_286.2+v_788+v_532+v_585.32+v_743.11+v_365+v_496.21+v_250.3+v_418+v_600+v_773+v_465+v_562.1+v_563+v_285+v_272.13+v_783+v_292.4+v_745+v_276.12+v_496+v_480+v_457.3+v_433.1+v_427.3+v_smoke+v_561.1+v_hypertension+v_760+v_721.1+v_586+v_head_injury+v_272.1+v_427.21+v_290.1+v_244.4+v_276.5+v_426.91+v_714.1+v_743.9+v_512.9+v_740.1+v_585.33+v_185+v_274.1+v_1

/home/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
INFO:dowhy.causal_estimator:INFO: Using Propensity Score Matching Estimator
INFO:dowhy.causal_estimator:b: v_alzheimer~v_296.2+v_280.1+v_722.6+v_292.3+v_401.22+v_Female+v_vascular_disease+v_443.9+v_386.9+v_789+v_495+v_411.2+v_366+v_433.21+v_782.3+v_heart_disease+v_300.1+v_433.8+v_350.2+v_916+v_530.11+v_585.31+v_425.1+v_585.1+v_339+v_332+v_452.2+v_obesity+v_261.4+v_290.3+v_1009+v_707.2+v_286.2+v_788+v_532+v_585.32+v_743.11+v_365+v_496.21+v_250.3+v_418+v_600+v_773+v_465+v_562.1+v_563+v_285+v_272.13+v_783+v_292.4+v_745+v_276.12+v_496+v_480+v_457.3+v_433.1+v_427.3+v_smoke+v_561.1+v_hypertension+v_760+v_721.1+v_586+v_head_injury+v_272.1+v_427.21+v_290.1+v_244.4+v_276.5+v_426.91+v_714.1+v_743.9+v_512.9+v_740.1+v_585.33+v_185+v_274.1+v_1

...

/home/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
INFO:dowhy.causal_estimator:INFO: Using Propensity Score Matching Estimator
INFO:dowhy.causal_estimator:b: v_alzheimer~v_296.2+v_280.1+v_722.6+v_292.3+v_401.22+v_Female+v_vascular_disease+v_443.9+v_386.9+v_789+v_495+v_411.2+v_366+v_433.21+v_782.3+v_heart_disease+v_300.1+v_433.8+v_350.2+v_916+v_530.11+v_585.31+v_425.1+v_585.1+v_339+v_332+v_452.2+v_obesity+v_261.4+v_290.3+v_1009+v_707.2+v_286.2+v_788+v_532+v_585.32+v_743.11+v_365+v_496.21+v_250.3+v_418+v_600+v_773+v_465+v_562.1+v_563+v_285+v_272.13+v_783+v_292.4+v_745+v_276.12+v_496+v_480+v_457.3+v_433.1+v_427.3+v_smoke+v_561.1+v_hypertension+v_760+v_721.1+v_586+v_head_injury+v_272.1+v_427.21+v_290.1+v_244.4+v_276.5+v_426.91+v_714.1+v_743.9+v_512.9+v_740.1+v_585.33+v_185+v_274.1+v_1

/home/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
INFO:dowhy.causal_estimator:INFO: Using Propensity Score Matching Estimator
INFO:dowhy.causal_estimator:b: v_alzheimer~v_296.2+v_280.1+v_722.6+v_292.3+v_401.22+v_Female+v_vascular_disease+v_443.9+v_386.9+v_789+v_495+v_411.2+v_366+v_433.21+v_782.3+v_heart_disease+v_300.1+v_433.8+v_350.2+v_916+v_530.11+v_585.31+v_425.1+v_585.1+v_339+v_332+v_452.2+v_obesity+v_261.4+v_290.3+v_1009+v_707.2+v_286.2+v_788+v_532+v_585.32+v_743.11+v_365+v_496.21+v_250.3+v_418+v_600+v_773+v_465+v_562.1+v_563+v_285+v_272.13+v_783+v_292.4+v_745+v_276.12+v_496+v_480+v_457.3+v_433.1+v_427.3+v_smoke+v_561.1+v_hypertension+v_760+v_721.1+v_586+v_head_injury+v_272.1+v_427.21+v_290.1+v_244.4+v_276.5+v_426.91+v_714.1+v_743.9+v_512.9+v_740.1+v_585.33+v_185+v_274.1+v_1

/home/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
INFO:dowhy.causal_estimator:INFO: Using Propensity Score Matching Estimator
INFO:dowhy.causal_estimator:b: v_alzheimer~v_296.2+v_280.1+v_722.6+v_292.3+v_401.22+v_Female+v_vascular_disease+v_443.9+v_386.9+v_789+v_495+v_411.2+v_366+v_433.21+v_782.3+v_heart_disease+v_300.1+v_433.8+v_350.2+v_916+v_530.11+v_585.31+v_425.1+v_585.1+v_339+v_332+v_452.2+v_obesity+v_261.4+v_290.3+v_1009+v_707.2+v_286.2+v_788+v_532+v_585.32+v_743.11+v_365+v_496.21+v_250.3+v_418+v_600+v_773+v_465+v_562.1+v_563+v_285+v_272.13+v_783+v_292.4+v_745+v_276.12+v_496+v_480+v_457.3+v_433.1+v_427.3+v_smoke+v_561.1+v_hypertension+v_760+v_721.1+v_586+v_head_injury+v_272.1+v_427.21+v_290.1+v_244.4+v_276.5+v_426.91+v_714.1+v_743.9+v_512.9+v_740.1+v_585.33+v_185+v_274.1+v_1

/home/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
INFO:dowhy.causal_estimator:INFO: Using Propensity Score Matching Estimator
INFO:dowhy.causal_estimator:b: v_alzheimer~v_296.2+v_280.1+v_722.6+v_292.3+v_401.22+v_Female+v_vascular_disease+v_443.9+v_386.9+v_789+v_495+v_411.2+v_366+v_433.21+v_782.3+v_heart_disease+v_300.1+v_433.8+v_350.2+v_916+v_530.11+v_585.31+v_425.1+v_585.1+v_339+v_332+v_452.2+v_obesity+v_261.4+v_290.3+v_1009+v_707.2+v_286.2+v_788+v_532+v_585.32+v_743.11+v_365+v_496.21+v_250.3+v_418+v_600+v_773+v_465+v_562.1+v_563+v_285+v_272.13+v_783+v_292.4+v_745+v_276.12+v_496+v_480+v_457.3+v_433.1+v_427.3+v_smoke+v_561.1+v_hypertension+v_760+v_721.1+v_586+v_head_injury+v_272.1+v_427.21+v_290.1+v_244.4+v_276.5+v_426.91+v_714.1+v_743.9+v_512.9+v_740.1+v_585.33+v_185+v_274.1+v_1

/home/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
INFO:dowhy.causal_estimator:INFO: Using Propensity Score Matching Estimator
INFO:dowhy.causal_estimator:b: v_alzheimer~v_296.2+v_280.1+v_722.6+v_292.3+v_401.22+v_Female+v_vascular_disease+v_443.9+v_386.9+v_789+v_495+v_411.2+v_366+v_433.21+v_782.3+v_heart_disease+v_300.1+v_433.8+v_350.2+v_916+v_530.11+v_585.31+v_425.1+v_585.1+v_339+v_332+v_452.2+v_obesity+v_261.4+v_290.3+v_1009+v_707.2+v_286.2+v_788+v_532+v_585.32+v_743.11+v_365+v_496.21+v_250.3+v_418+v_600+v_773+v_465+v_562.1+v_563+v_285+v_272.13+v_783+v_292.4+v_745+v_276.12+v_496+v_480+v_457.3+v_433.1+v_427.3+v_smoke+v_561.1+v_hypertension+v_760+v_721.1+v_586+v_head_injury+v_272.1+v_427.21+v_290.1+v_244.4+v_276.5+v_426.91+v_714.1+v_743.9+v_512.9+v_740.1+v_585.33+v_185+v_274.1+v_1

In [26]:
estimate=model.estimate_effect(identified_estimand,
                              method_name='backdoor.propensity_score_weighting',
                               method_params={"weighting_scheme":"ips_normalized_weight"},
                              target_units='atc')

INFO:dowhy.causal_estimator:INFO: Using Propensity Score Weighting Estimator
INFO:dowhy.causal_estimator:b: v_alzheimer~v_296.2+v_280.1+v_722.6+v_292.3+v_401.22+v_Female+v_vascular_disease+v_443.9+v_386.9+v_789+v_495+v_411.2+v_366+v_433.21+v_782.3+v_heart_disease+v_300.1+v_433.8+v_350.2+v_916+v_530.11+v_585.31+v_425.1+v_585.1+v_339+v_332+v_452.2+v_obesity+v_261.4+v_290.3+v_1009+v_707.2+v_286.2+v_788+v_532+v_585.32+v_743.11+v_365+v_496.21+v_250.3+v_418+v_600+v_773+v_465+v_562.1+v_563+v_285+v_272.13+v_783+v_292.4+v_745+v_276.12+v_496+v_480+v_457.3+v_433.1+v_427.3+v_smoke+v_561.1+v_hypertension+v_760+v_721.1+v_586+v_head_injury+v_272.1+v_427.21+v_290.1+v_244.4+v_276.5+v_426.91+v_714.1+v_743.9+v_512.9+v_740.1+v_585.33+v_185+v_274.1+v_1002+v_585.3+v_512.8+v_707.1+v_591+v_272.11+v_395.1+v_174.11+v_395.2+v_285.21+v_401.1+v_250.2+v_428.1+v_798+v_785+v_716.9+v_426.7+v_740.9+v_diabetes+v_318+v_452+v_1010+v_512.7+v_433.31+v_276.14+v_208+v_411.4+v_153.2
/home/anaconda3/lib/python3.7/site-packages/

In [18]:
print(estimate)

*** Causal Estimate ***

## Identified estimand
Estimand type: nonparametric-ate
### Estimand : 1
Estimand name: backdoor
Estimand expression:
    d                                                                         
──────────(Expectation(v_alzheimer|v_280.1,v_722.6,v_292.3,v_401.22,v_Female,v
d[v_296.2]                                                                    

                                                                              
_vascular_disease,v_443.9,v_386.9,v_789,v_495,v_411.2,v_366,v_433.21,v_782.3,v
                                                                              

                                                                              
_heart_disease,v_300.1,v_433.8,v_350.2,v_916,v_530.11,v_585.31,v_425.1,v_585.1
                                                                              

                                                                              
,v_339,v_332,v_452.2,v_obesity,v_261.4,v_290.3,v_1009,v_707.2,v_

In [27]:
estimate.get_confidence_intervals()

INFO:dowhy.causal_estimator:INFO: The sample size: 13460
INFO:dowhy.causal_estimator:INFO: The number of simulations: 100
INFO:dowhy.causal_estimator:INFO: Using Propensity Score Weighting Estimator
INFO:dowhy.causal_estimator:b: v_alzheimer~v_296.2+v_280.1+v_722.6+v_292.3+v_401.22+v_Female+v_vascular_disease+v_443.9+v_386.9+v_789+v_495+v_411.2+v_366+v_433.21+v_782.3+v_heart_disease+v_300.1+v_433.8+v_350.2+v_916+v_530.11+v_585.31+v_425.1+v_585.1+v_339+v_332+v_452.2+v_obesity+v_261.4+v_290.3+v_1009+v_707.2+v_286.2+v_788+v_532+v_585.32+v_743.11+v_365+v_496.21+v_250.3+v_418+v_600+v_773+v_465+v_562.1+v_563+v_285+v_272.13+v_783+v_292.4+v_745+v_276.12+v_496+v_480+v_457.3+v_433.1+v_427.3+v_smoke+v_561.1+v_hypertension+v_760+v_721.1+v_586+v_head_injury+v_272.1+v_427.21+v_290.1+v_244.4+v_276.5+v_426.91+v_714.1+v_743.9+v_512.9+v_740.1+v_585.33+v_185+v_274.1+v_1002+v_585.3+v_512.8+v_707.1+v_591+v_272.11+v_395.1+v_174.11+v_395.2+v_285.21+v_401.1+v_250.2+v_428.1+v_798+v_785+v_716.9+v_426.7+v_740.9+

/home/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
INFO:dowhy.causal_estimator:INFO: Using Propensity Score Weighting Estimator
INFO:dowhy.causal_estimator:b: v_alzheimer~v_296.2+v_280.1+v_722.6+v_292.3+v_401.22+v_Female+v_vascular_disease+v_443.9+v_386.9+v_789+v_495+v_411.2+v_366+v_433.21+v_782.3+v_heart_disease+v_300.1+v_433.8+v_350.2+v_916+v_530.11+v_585.31+v_425.1+v_585.1+v_339+v_332+v_452.2+v_obesity+v_261.4+v_290.3+v_1009+v_707.2+v_286.2+v_788+v_532+v_585.32+v_743.11+v_365+v_496.21+v_250.3+v_418+v_600+v_773+v_465+v_562.1+v_563+v_285+v_272.13+v_783+v_292.4+v_745+v_276.12+v_496+v_480+v_457.3+v_433.1+v_427.3+v_smoke+v_561.1+v_hypertension+v_760+v_721.1+v_586+v_head_injury+v_272.1+v_427.21+v_290.1+v_244.4+v_276.5+v_426.91+v_714.1+v_743.9+v_512.9+v_740.1+v_585.33+v_185+v_274.1+v_

/home/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
INFO:dowhy.causal_estimator:INFO: Using Propensity Score Weighting Estimator
INFO:dowhy.causal_estimator:b: v_alzheimer~v_296.2+v_280.1+v_722.6+v_292.3+v_401.22+v_Female+v_vascular_disease+v_443.9+v_386.9+v_789+v_495+v_411.2+v_366+v_433.21+v_782.3+v_heart_disease+v_300.1+v_433.8+v_350.2+v_916+v_530.11+v_585.31+v_425.1+v_585.1+v_339+v_332+v_452.2+v_obesity+v_261.4+v_290.3+v_1009+v_707.2+v_286.2+v_788+v_532+v_585.32+v_743.11+v_365+v_496.21+v_250.3+v_418+v_600+v_773+v_465+v_562.1+v_563+v_285+v_272.13+v_783+v_292.4+v_745+v_276.12+v_496+v_480+v_457.3+v_433.1+v_427.3+v_smoke+v_561.1+v_hypertension+v_760+v_721.1+v_586+v_head_injury+v_272.1+v_427.21+v_290.1+v_244.4+v_276.5+v_426.91+v_714.1+v_743.9+v_512.9+v_740.1+v_585.33+v_185+v_274.1+v_

/home/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
INFO:dowhy.causal_estimator:INFO: Using Propensity Score Weighting Estimator
INFO:dowhy.causal_estimator:b: v_alzheimer~v_296.2+v_280.1+v_722.6+v_292.3+v_401.22+v_Female+v_vascular_disease+v_443.9+v_386.9+v_789+v_495+v_411.2+v_366+v_433.21+v_782.3+v_heart_disease+v_300.1+v_433.8+v_350.2+v_916+v_530.11+v_585.31+v_425.1+v_585.1+v_339+v_332+v_452.2+v_obesity+v_261.4+v_290.3+v_1009+v_707.2+v_286.2+v_788+v_532+v_585.32+v_743.11+v_365+v_496.21+v_250.3+v_418+v_600+v_773+v_465+v_562.1+v_563+v_285+v_272.13+v_783+v_292.4+v_745+v_276.12+v_496+v_480+v_457.3+v_433.1+v_427.3+v_smoke+v_561.1+v_hypertension+v_760+v_721.1+v_586+v_head_injury+v_272.1+v_427.21+v_290.1+v_244.4+v_276.5+v_426.91+v_714.1+v_743.9+v_512.9+v_740.1+v_585.33+v_185+v_274.1+v_

/home/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
INFO:dowhy.causal_estimator:INFO: Using Propensity Score Weighting Estimator
INFO:dowhy.causal_estimator:b: v_alzheimer~v_296.2+v_280.1+v_722.6+v_292.3+v_401.22+v_Female+v_vascular_disease+v_443.9+v_386.9+v_789+v_495+v_411.2+v_366+v_433.21+v_782.3+v_heart_disease+v_300.1+v_433.8+v_350.2+v_916+v_530.11+v_585.31+v_425.1+v_585.1+v_339+v_332+v_452.2+v_obesity+v_261.4+v_290.3+v_1009+v_707.2+v_286.2+v_788+v_532+v_585.32+v_743.11+v_365+v_496.21+v_250.3+v_418+v_600+v_773+v_465+v_562.1+v_563+v_285+v_272.13+v_783+v_292.4+v_745+v_276.12+v_496+v_480+v_457.3+v_433.1+v_427.3+v_smoke+v_561.1+v_hypertension+v_760+v_721.1+v_586+v_head_injury+v_272.1+v_427.21+v_290.1+v_244.4+v_276.5+v_426.91+v_714.1+v_743.9+v_512.9+v_740.1+v_585.33+v_185+v_274.1+v_

...

/home/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
INFO:dowhy.causal_estimator:INFO: Using Propensity Score Weighting Estimator
INFO:dowhy.causal_estimator:b: v_alzheimer~v_296.2+v_280.1+v_722.6+v_292.3+v_401.22+v_Female+v_vascular_disease+v_443.9+v_386.9+v_789+v_495+v_411.2+v_366+v_433.21+v_782.3+v_heart_disease+v_300.1+v_433.8+v_350.2+v_916+v_530.11+v_585.31+v_425.1+v_585.1+v_339+v_332+v_452.2+v_obesity+v_261.4+v_290.3+v_1009+v_707.2+v_286.2+v_788+v_532+v_585.32+v_743.11+v_365+v_496.21+v_250.3+v_418+v_600+v_773+v_465+v_562.1+v_563+v_285+v_272.13+v_783+v_292.4+v_745+v_276.12+v_496+v_480+v_457.3+v_433.1+v_427.3+v_smoke+v_561.1+v_hypertension+v_760+v_721.1+v_586+v_head_injury+v_272.1+v_427.21+v_290.1+v_244.4+v_276.5+v_426.91+v_714.1+v_743.9+v_512.9+v_740.1+v_585.33+v_185+v_274.1+v_

/home/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
INFO:dowhy.causal_estimator:INFO: Using Propensity Score Weighting Estimator
INFO:dowhy.causal_estimator:b: v_alzheimer~v_296.2+v_280.1+v_722.6+v_292.3+v_401.22+v_Female+v_vascular_disease+v_443.9+v_386.9+v_789+v_495+v_411.2+v_366+v_433.21+v_782.3+v_heart_disease+v_300.1+v_433.8+v_350.2+v_916+v_530.11+v_585.31+v_425.1+v_585.1+v_339+v_332+v_452.2+v_obesity+v_261.4+v_290.3+v_1009+v_707.2+v_286.2+v_788+v_532+v_585.32+v_743.11+v_365+v_496.21+v_250.3+v_418+v_600+v_773+v_465+v_562.1+v_563+v_285+v_272.13+v_783+v_292.4+v_745+v_276.12+v_496+v_480+v_457.3+v_433.1+v_427.3+v_smoke+v_561.1+v_hypertension+v_760+v_721.1+v_586+v_head_injury+v_272.1+v_427.21+v_290.1+v_244.4+v_276.5+v_426.91+v_714.1+v_743.9+v_512.9+v_740.1+v_585.33+v_185+v_274.1+v_

/home/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
INFO:dowhy.causal_estimator:INFO: Using Propensity Score Weighting Estimator
INFO:dowhy.causal_estimator:b: v_alzheimer~v_296.2+v_280.1+v_722.6+v_292.3+v_401.22+v_Female+v_vascular_disease+v_443.9+v_386.9+v_789+v_495+v_411.2+v_366+v_433.21+v_782.3+v_heart_disease+v_300.1+v_433.8+v_350.2+v_916+v_530.11+v_585.31+v_425.1+v_585.1+v_339+v_332+v_452.2+v_obesity+v_261.4+v_290.3+v_1009+v_707.2+v_286.2+v_788+v_532+v_585.32+v_743.11+v_365+v_496.21+v_250.3+v_418+v_600+v_773+v_465+v_562.1+v_563+v_285+v_272.13+v_783+v_292.4+v_745+v_276.12+v_496+v_480+v_457.3+v_433.1+v_427.3+v_smoke+v_561.1+v_hypertension+v_760+v_721.1+v_586+v_head_injury+v_272.1+v_427.21+v_290.1+v_244.4+v_276.5+v_426.91+v_714.1+v_743.9+v_512.9+v_740.1+v_585.33+v_185+v_274.1+v_

/home/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
INFO:dowhy.causal_estimator:INFO: Using Propensity Score Weighting Estimator
INFO:dowhy.causal_estimator:b: v_alzheimer~v_296.2+v_280.1+v_722.6+v_292.3+v_401.22+v_Female+v_vascular_disease+v_443.9+v_386.9+v_789+v_495+v_411.2+v_366+v_433.21+v_782.3+v_heart_disease+v_300.1+v_433.8+v_350.2+v_916+v_530.11+v_585.31+v_425.1+v_585.1+v_339+v_332+v_452.2+v_obesity+v_261.4+v_290.3+v_1009+v_707.2+v_286.2+v_788+v_532+v_585.32+v_743.11+v_365+v_496.21+v_250.3+v_418+v_600+v_773+v_465+v_562.1+v_563+v_285+v_272.13+v_783+v_292.4+v_745+v_276.12+v_496+v_480+v_457.3+v_433.1+v_427.3+v_smoke+v_561.1+v_hypertension+v_760+v_721.1+v_586+v_head_injury+v_272.1+v_427.21+v_290.1+v_244.4+v_276.5+v_426.91+v_714.1+v_743.9+v_512.9+v_740.1+v_585.33+v_185+v_274.1+v_

/home/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
INFO:dowhy.causal_estimator:INFO: Using Propensity Score Weighting Estimator
INFO:dowhy.causal_estimator:b: v_alzheimer~v_296.2+v_280.1+v_722.6+v_292.3+v_401.22+v_Female+v_vascular_disease+v_443.9+v_386.9+v_789+v_495+v_411.2+v_366+v_433.21+v_782.3+v_heart_disease+v_300.1+v_433.8+v_350.2+v_916+v_530.11+v_585.31+v_425.1+v_585.1+v_339+v_332+v_452.2+v_obesity+v_261.4+v_290.3+v_1009+v_707.2+v_286.2+v_788+v_532+v_585.32+v_743.11+v_365+v_496.21+v_250.3+v_418+v_600+v_773+v_465+v_562.1+v_563+v_285+v_272.13+v_783+v_292.4+v_745+v_276.12+v_496+v_480+v_457.3+v_433.1+v_427.3+v_smoke+v_561.1+v_hypertension+v_760+v_721.1+v_586+v_head_injury+v_272.1+v_427.21+v_290.1+v_244.4+v_276.5+v_426.91+v_714.1+v_743.9+v_512.9+v_740.1+v_585.33+v_185+v_274.1+v_

In [28]:
estimate=model.estimate_effect(identified_estimand,
                              method_name='backdoor.propensity_score_stratification', 
                               target_units='atc')

INFO:dowhy.causal_estimator:INFO: Using Propensity Score Stratification Estimator
INFO:dowhy.causal_estimator:b: v_alzheimer~v_296.2+v_280.1+v_722.6+v_292.3+v_401.22+v_Female+v_vascular_disease+v_443.9+v_386.9+v_789+v_495+v_411.2+v_366+v_433.21+v_782.3+v_heart_disease+v_300.1+v_433.8+v_350.2+v_916+v_530.11+v_585.31+v_425.1+v_585.1+v_339+v_332+v_452.2+v_obesity+v_261.4+v_290.3+v_1009+v_707.2+v_286.2+v_788+v_532+v_585.32+v_743.11+v_365+v_496.21+v_250.3+v_418+v_600+v_773+v_465+v_562.1+v_563+v_285+v_272.13+v_783+v_292.4+v_745+v_276.12+v_496+v_480+v_457.3+v_433.1+v_427.3+v_smoke+v_561.1+v_hypertension+v_760+v_721.1+v_586+v_head_injury+v_272.1+v_427.21+v_290.1+v_244.4+v_276.5+v_426.91+v_714.1+v_743.9+v_512.9+v_740.1+v_585.33+v_185+v_274.1+v_1002+v_585.3+v_512.8+v_707.1+v_591+v_272.11+v_395.1+v_174.11+v_395.2+v_285.21+v_401.1+v_250.2+v_428.1+v_798+v_785+v_716.9+v_426.7+v_740.9+v_diabetes+v_318+v_452+v_1010+v_512.7+v_433.31+v_276.14+v_208+v_411.4+v_153.2
/home/anaconda3/lib/python3.7/site-pack

In [29]:
estimate.get_confidence_intervals()

INFO:dowhy.causal_estimator:INFO: The sample size: 13460
INFO:dowhy.causal_estimator:INFO: The number of simulations: 100
INFO:dowhy.causal_estimator:INFO: Using Propensity Score Stratification Estimator
INFO:dowhy.causal_estimator:b: v_alzheimer~v_296.2+v_280.1+v_722.6+v_292.3+v_401.22+v_Female+v_vascular_disease+v_443.9+v_386.9+v_789+v_495+v_411.2+v_366+v_433.21+v_782.3+v_heart_disease+v_300.1+v_433.8+v_350.2+v_916+v_530.11+v_585.31+v_425.1+v_585.1+v_339+v_332+v_452.2+v_obesity+v_261.4+v_290.3+v_1009+v_707.2+v_286.2+v_788+v_532+v_585.32+v_743.11+v_365+v_496.21+v_250.3+v_418+v_600+v_773+v_465+v_562.1+v_563+v_285+v_272.13+v_783+v_292.4+v_745+v_276.12+v_496+v_480+v_457.3+v_433.1+v_427.3+v_smoke+v_561.1+v_hypertension+v_760+v_721.1+v_586+v_head_injury+v_272.1+v_427.21+v_290.1+v_244.4+v_276.5+v_426.91+v_714.1+v_743.9+v_512.9+v_740.1+v_585.33+v_185+v_274.1+v_1002+v_585.3+v_512.8+v_707.1+v_591+v_272.11+v_395.1+v_174.11+v_395.2+v_285.21+v_401.1+v_250.2+v_428.1+v_798+v_785+v_716.9+v_426.7+v_7

/home/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/home/anaconda3/lib/python3.7/site-packages/pandas/core/computation/expressions.py:205: UserWarning: evaluating in Python space because the '*' operator is not supported by numexpr for the bool dtype, use '&' instead
  f"evaluating in Python space because the {repr(op_str)} "
/home/anaconda3/lib/python3.7/site-packages/dowhy/causal_estimators/propensity_score_stratification_estimator.py:62: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.
  weighted_outcomes.columns = ["_".join(x) for x in weighted_outcomes.columns.ravel()]
INFO:dowhy.causal_estimator:INFO: Using Propensity Score Stratification Estimator
INFO:dowhy.causal_estimator:b: v_alzheimer~v_296.2+v_280.1+v_722

INFO:dowhy.causal_estimator:b: v_alzheimer~v_296.2+v_280.1+v_722.6+v_292.3+v_401.22+v_Female+v_vascular_disease+v_443.9+v_386.9+v_789+v_495+v_411.2+v_366+v_433.21+v_782.3+v_heart_disease+v_300.1+v_433.8+v_350.2+v_916+v_530.11+v_585.31+v_425.1+v_585.1+v_339+v_332+v_452.2+v_obesity+v_261.4+v_290.3+v_1009+v_707.2+v_286.2+v_788+v_532+v_585.32+v_743.11+v_365+v_496.21+v_250.3+v_418+v_600+v_773+v_465+v_562.1+v_563+v_285+v_272.13+v_783+v_292.4+v_745+v_276.12+v_496+v_480+v_457.3+v_433.1+v_427.3+v_smoke+v_561.1+v_hypertension+v_760+v_721.1+v_586+v_head_injury+v_272.1+v_427.21+v_290.1+v_244.4+v_276.5+v_426.91+v_714.1+v_743.9+v_512.9+v_740.1+v_585.33+v_185+v_274.1+v_1002+v_585.3+v_512.8+v_707.1+v_591+v_272.11+v_395.1+v_174.11+v_395.2+v_285.21+v_401.1+v_250.2+v_428.1+v_798+v_785+v_716.9+v_426.7+v_740.9+v_diabetes+v_318+v_452+v_1010+v_512.7+v_433.31+v_276.14+v_208+v_411.4+v_153.2
/home/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was 

/home/anaconda3/lib/python3.7/site-packages/pandas/core/computation/expressions.py:205: UserWarning: evaluating in Python space because the '*' operator is not supported by numexpr for the bool dtype, use '&' instead
  f"evaluating in Python space because the {repr(op_str)} "
/home/anaconda3/lib/python3.7/site-packages/dowhy/causal_estimators/propensity_score_stratification_estimator.py:62: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.
  weighted_outcomes.columns = ["_".join(x) for x in weighted_outcomes.columns.ravel()]
INFO:dowhy.causal_estimator:INFO: Using Propensity Score Stratification Estimator
INFO:dowhy.causal_estimator:b: v_alzheimer~v_296.2+v_280.1+v_722.6+v_292.3+v_401.22+v_Female+v_vascular_disease+v_443.9+v_386.9+v_789+v_495+v_411.2+v_366+v_433.21+v_782.3+v_heart_disease+v_300.1+v_433.8+v_350.2+v_916+v_530.11+v_585.31+v_425.1+v_585.1+v_339+v_332+v_452.2+v_obesity+v_261.4+v_290.3+v_1009+v_707.2+v_286.2+v_78

/home/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/home/anaconda3/lib/python3.7/site-packages/pandas/core/computation/expressions.py:205: UserWarning: evaluating in Python space because the '*' operator is not supported by numexpr for the bool dtype, use '&' instead
  f"evaluating in Python space because the {repr(op_str)} "
/home/anaconda3/lib/python3.7/site-packages/dowhy/causal_estimators/propensity_score_stratification_estimator.py:62: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.
  weighted_outcomes.columns = ["_".join(x) for x in weighted_outcomes.columns.ravel()]
INFO:dowhy.causal_estimator:INFO: Using Propensity Score Stratification Estimator
INFO:dowhy.causal_estimator:b: v_alzheimer~v_296.2+v_280.1+v_722

...

INFO:dowhy.causal_estimator:b: v_alzheimer~v_296.2+v_280.1+v_722.6+v_292.3+v_401.22+v_Female+v_vascular_disease+v_443.9+v_386.9+v_789+v_495+v_411.2+v_366+v_433.21+v_782.3+v_heart_disease+v_300.1+v_433.8+v_350.2+v_916+v_530.11+v_585.31+v_425.1+v_585.1+v_339+v_332+v_452.2+v_obesity+v_261.4+v_290.3+v_1009+v_707.2+v_286.2+v_788+v_532+v_585.32+v_743.11+v_365+v_496.21+v_250.3+v_418+v_600+v_773+v_465+v_562.1+v_563+v_285+v_272.13+v_783+v_292.4+v_745+v_276.12+v_496+v_480+v_457.3+v_433.1+v_427.3+v_smoke+v_561.1+v_hypertension+v_760+v_721.1+v_586+v_head_injury+v_272.1+v_427.21+v_290.1+v_244.4+v_276.5+v_426.91+v_714.1+v_743.9+v_512.9+v_740.1+v_585.33+v_185+v_274.1+v_1002+v_585.3+v_512.8+v_707.1+v_591+v_272.11+v_395.1+v_174.11+v_395.2+v_285.21+v_401.1+v_250.2+v_428.1+v_798+v_785+v_716.9+v_426.7+v_740.9+v_diabetes+v_318+v_452+v_1010+v_512.7+v_433.31+v_276.14+v_208+v_411.4+v_153.2
/home/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was 

/home/anaconda3/lib/python3.7/site-packages/pandas/core/computation/expressions.py:205: UserWarning: evaluating in Python space because the '*' operator is not supported by numexpr for the bool dtype, use '&' instead
  f"evaluating in Python space because the {repr(op_str)} "
/home/anaconda3/lib/python3.7/site-packages/dowhy/causal_estimators/propensity_score_stratification_estimator.py:62: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.
  weighted_outcomes.columns = ["_".join(x) for x in weighted_outcomes.columns.ravel()]
INFO:dowhy.causal_estimator:INFO: Using Propensity Score Stratification Estimator
INFO:dowhy.causal_estimator:b: v_alzheimer~v_296.2+v_280.1+v_722.6+v_292.3+v_401.22+v_Female+v_vascular_disease+v_443.9+v_386.9+v_789+v_495+v_411.2+v_366+v_433.21+v_782.3+v_heart_disease+v_300.1+v_433.8+v_350.2+v_916+v_530.11+v_585.31+v_425.1+v_585.1+v_339+v_332+v_452.2+v_obesity+v_261.4+v_290.3+v_1009+v_707.2+v_286.2+v_78

/home/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/home/anaconda3/lib/python3.7/site-packages/pandas/core/computation/expressions.py:205: UserWarning: evaluating in Python space because the '*' operator is not supported by numexpr for the bool dtype, use '&' instead
  f"evaluating in Python space because the {repr(op_str)} "
/home/anaconda3/lib/python3.7/site-packages/dowhy/causal_estimators/propensity_score_stratification_estimator.py:62: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.
  weighted_outcomes.columns = ["_".join(x) for x in weighted_outcomes.columns.ravel()]
INFO:dowhy.causal_estimator:INFO: Using Propensity Score Stratification Estimator
INFO:dowhy.causal_estimator:b: v_alzheimer~v_296.2+v_280.1+v_722

INFO:dowhy.causal_estimator:b: v_alzheimer~v_296.2+v_280.1+v_722.6+v_292.3+v_401.22+v_Female+v_vascular_disease+v_443.9+v_386.9+v_789+v_495+v_411.2+v_366+v_433.21+v_782.3+v_heart_disease+v_300.1+v_433.8+v_350.2+v_916+v_530.11+v_585.31+v_425.1+v_585.1+v_339+v_332+v_452.2+v_obesity+v_261.4+v_290.3+v_1009+v_707.2+v_286.2+v_788+v_532+v_585.32+v_743.11+v_365+v_496.21+v_250.3+v_418+v_600+v_773+v_465+v_562.1+v_563+v_285+v_272.13+v_783+v_292.4+v_745+v_276.12+v_496+v_480+v_457.3+v_433.1+v_427.3+v_smoke+v_561.1+v_hypertension+v_760+v_721.1+v_586+v_head_injury+v_272.1+v_427.21+v_290.1+v_244.4+v_276.5+v_426.91+v_714.1+v_743.9+v_512.9+v_740.1+v_585.33+v_185+v_274.1+v_1002+v_585.3+v_512.8+v_707.1+v_591+v_272.11+v_395.1+v_174.11+v_395.2+v_285.21+v_401.1+v_250.2+v_428.1+v_798+v_785+v_716.9+v_426.7+v_740.9+v_diabetes+v_318+v_452+v_1010+v_512.7+v_433.31+v_276.14+v_208+v_411.4+v_153.2
/home/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was 

/home/anaconda3/lib/python3.7/site-packages/pandas/core/computation/expressions.py:205: UserWarning: evaluating in Python space because the '*' operator is not supported by numexpr for the bool dtype, use '&' instead
  f"evaluating in Python space because the {repr(op_str)} "
/home/anaconda3/lib/python3.7/site-packages/dowhy/causal_estimators/propensity_score_stratification_estimator.py:62: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.
  weighted_outcomes.columns = ["_".join(x) for x in weighted_outcomes.columns.ravel()]
INFO:dowhy.causal_estimator:INFO: Using Propensity Score Stratification Estimator
INFO:dowhy.causal_estimator:b: v_alzheimer~v_296.2+v_280.1+v_722.6+v_292.3+v_401.22+v_Female+v_vascular_disease+v_443.9+v_386.9+v_789+v_495+v_411.2+v_366+v_433.21+v_782.3+v_heart_disease+v_300.1+v_433.8+v_350.2+v_916+v_530.11+v_585.31+v_425.1+v_585.1+v_339+v_332+v_452.2+v_obesity+v_261.4+v_290.3+v_1009+v_707.2+v_286.2+v_78

## Refute

Confirm the estimated treatment

In [ ]:
res_random=model.refute_estimate(identified_estimand, estimate, method_name="random_common_cause")
print(res_random)

In [ ]:
res_placebo=model.refute_estimate(identified_estimand, estimate,
        method_name="placebo_treatment_refuter", placebo_type="permute",)
print(res_placebo)